In [2]:
%reload_ext autoreload
%autoreload 2

import os
import pprint
import shutil
import pymongo

from climatools.cliradlw import (setup, pipeline)
import climatools.cliradlw.runrecord as runrecord_clirad
import climatools.cliradlw.utils as utils_clirad

from climatools.lblnew import setup_overlap
from climatools.lblnew import setup_bestfit
import climatools.lblnew.pipeline as pipeline_lblnew
import climatools.lblnew.bestfit_params as bestfit
import climatools.lblnew.runrecord_bestfit as runrecord_bestfit
import climatools.lblnew.runrecord_overlap as runrecord_overlap

In [3]:
client = pymongo.MongoClient('localhost', 27017)

In [4]:
client.cliradnew.lw.count()

159

In [9]:
ATMPROS = [('mls', 294), ('saw', 257), ('trp', 300)]

In [52]:
# EDIT HERE TO START

def params_clirad_nonoverlap():
    '''
    Define clirad-lw input parameter dictionaries
    '''
    commitnumber = '1013f91'
    molecule = {'co2': 400e-6}
    bands = [4,] 
    
    return [{'commitnumber': commitnumber,
             'molecule': molecule,
             'band': [band],
             'atmpro': atmpro, 'tsfc': tsfc}
            for band in bands
            for atmpro, tsfc in ATMPROS]

In [33]:
def param_clirad2lblnewbestfit(param=None, commitnumber=None):
    '''
    Concert clirad param to lblnew-bestfit partial param
    
    Parameters
    ----------
    param: dict
        clirad's param dictionary
    commitnumber: str
        Git commit number of the latest lblnew-bestfit version
    '''
    molecule, conc = list(param['molecule'].items())[0]
    band = param['band'][0]
    band = utils_clirad.mapband_new2old()[band]
    atmpro = param['atmpro']
    tsfc = param['tsfc']
    return {'commitnumber': commitnumber,
            'molecule': molecule, 'conc': conc,
            'band': band, 
            'atmpro': atmpro, 'tsfc': tsfc}

In [53]:
def to_lblnewparam(param=None, commitnumber=None):
    '''
    Make query for lblnew-bestfit runs' database from 
    a given clirad-lw param
    '''
    # Unpack clirad-lw's param
    molecule, conc = list(param['molecule'].items())[0]
    band = param['band'][0]
    band = utils_clirad.mapband_new2old()[band]
    atmpro = param['atmpro']
    tsfc = param['tsfc']
    
    # Get bestfit parameter values for (molecule, band)
    param_best = bestfit.kdist_params(molecule=molecule, band=band)
    
    param_best['atmpro'] = atmpro
    param_best['tsfc'] = tsfc
    param_best['option_compute_ktable'] = 0
    param_best['option_compute_btable'] = 0
    return param_best

def make_query(param=None)   :
    return {f'param.{n}': v for n, v in param.items()}

def get_parampairs():
    pairs = []
    for p in params_clirad_nonoverlap():
        p_lblnew = to_lblnewparam(param=p)
        q = make_query(param=p_lblnew)
        ndocs = client.lblnew.bestfit_lw.count(q)
        if ndocs: pairs.append((p, p_lblnew))
    return pairs

In [60]:
def script_clirad_lblnew():
    '''
    Script to run clirad and then plot results against the 
    corresponding lblnew case.  This needs both input parameter
    dictionaries from clirad and lblnew.
    '''
    ps = get_parampairs()
    gprocs = pipeline.pipeline_ipynb2git(parampairs=ps, setup=setup)

In [65]:
script_clirad_lblnew()

{'atmpro': 'mls',
 'band': [4],
 'commitnumber': '1013f91',
 'molecule': {'co2': 0.0004},
 'tsfc': 294}
{'atmpro': 'saw',
 'band': [4],
 'commitnumber': '1013f91',
 'molecule': {'co2': 0.0004},
 'tsfc': 257}
{'atmpro': 'trp',
 'band': [4],
 'commitnumber': '1013f91',
 'molecule': {'co2': 0.0004},
 'tsfc': 300}
{'atmpro': 'mls',
 'band': [4],
 'commitnumber': '1013f91',
 'molecule': {'co2': 0.0004},
 'tsfc': 294}
{'atmpro': 'saw',
 'band': [4],
 'commitnumber': '1013f91',
 'molecule': {'co2': 0.0004},
 'tsfc': 257}
{'atmpro': 'trp',
 'band': [4],
 'commitnumber': '1013f91',
 'molecule': {'co2': 0.0004},
 'tsfc': 300}



In [66]:
for param, _ in get_parampairs():
    print(pipeline.nbviewer_url(param=param, setup=setup))
    print()


http://nbviewer.jupyter.org/github/qap/analysis_-_new_kdist_param/blob/master/clirad/co2_0.0004/band_4/atmpro_mls/cliradlw_1013f91/results_cliradlw.ipynb

http://nbviewer.jupyter.org/github/qap/analysis_-_new_kdist_param/blob/master/clirad/co2_0.0004/band_4/atmpro_saw/cliradlw_1013f91/results_cliradlw.ipynb

http://nbviewer.jupyter.org/github/qap/analysis_-_new_kdist_param/blob/master/clirad/co2_0.0004/band_4/atmpro_trp/cliradlw_1013f91/results_cliradlw.ipynb

